In [29]:
from pyspark import SparkConf, SparkContext
import os

In [30]:
conf = SparkConf().setMaster("local").setAppName("matrix_multiplication")
sc = SparkContext(conf=conf)

In [ ]:
data = sc.textFile("/Users/haoweichung/Desktop/清大/大三上/data_mining/HW1/500input.txt")
f = open("/Users/haoweichung/Desktop/清大/大三上/data_mining/HW1/Output.txt", "a")


matM = data.filter(lambda x: x.split(',')[0] == 'M').map(lambda x: ( x.split(',')[2],( x.split(',')[0],  x.split(',')[1],  x.split(',')[3])))
matN = data.filter(lambda x: x.split(',')[0] == 'N').map(lambda x: ( x.split(',')[1],( x.split(',')[0],  x.split(',')[2],  x.split(',')[3])))
join_data = matM.join(matN).map(lambda x: ((x[1][0][1], x[1][1][1]), int(x[1][0][2]) * int(x[1][1][2]))).reduceByKey(lambda x,y: x+y)
result = join_data.sortBy(lambda x: (int(x[0][0]), int(x[0][1]))).collect()

for out in result:
    f.write('%s,%s,%d\n'%(out[0][0],out[0][1],out[1]))
    print('%s,%s,%d\n'%(out[0][0],out[0][1],out[1]))
f.close()   
sc.stop()  





# MDA_HW1_Matrix_Multiplication

首先先用filter將兩個矩陣分開成 MatM, MatN。並把兩組資料map成:
$$ (j ,(M,i,{M}_{ij})) $$
$$ (j ,(N,k,{N}_{jk})) $$

接者，把兩組data join在一起，我們可以得到:

$$ (j ,(('M',i,{M}_{ij}),('N',k,{N}_{jk})) $$

接者在重新做一次map，把(i,j)當key，元素相乘當作value:

$$ ((i,k), {M}_{ij}{N}_{jk}) $$

接者做一次reduce把所有擁有相同：
$$  (i,k) $$
key的pair的value全部相加。最後得到矩陣相乘的結果。


